In [ ]:
!cat data/TREC/trec05p-1/README.txt
!python3 --version

In [ ]:
import os
from html.parser import HTMLParser


class MyHTMLParser(HTMLParser):
    def __init__ (self):
        super(MyHTMLParser, self).__init__()
        self.current_email = ""
    def handle_starttag(self, tag, attrs):
        pass
    def handle_endtag(self, tag):
        pass
    def handle_data(self, data):
        self.current_email += data + " "

In [ ]:
def extract_metadata (encoded_message, corpus, collection):
    sender = ""
    content_type = ""
    boundary = None
    body_index = None
    split_indices = []
    
    for i, line in enumerate(encoded_message):
        sline = line.decode('latin-1')
        if str.encode("From: ") in line:
            sender = sline[6:].strip()
        if content_type == '' and str.encode("Content-Type: ") in line:
            content_type = sline[14:sline.find(';')]
        if 'boundary' in sline and sline.find('=') != -1:
            boundary = sline[sline.find('=')+1:][1:-2]
            continue
        if boundary and boundary in sline:
            split_indices.append(i)
        if not body_index and sline.strip() == "":
            body_index = i
    
    if len(split_indices) == 0:
        split_indices.append(body_index)
    split_indices.append(len(encoded_message)-1)
    
#     print("------------", corpus, collection)
#     print(sender)
#     print(content_type)
#     print(boundary)
#     print(body_index)
#     print(split_indices)
    
    return (sender, content_type, boundary, split_indices)
    
            

In [ ]:
data_dir = './data/TREC/trec05p-1/data/'
senders = []
dir_limit = 1

for corpus in sorted(os.listdir(data_dir)):
    file_limit = 100
    for collection in sorted(os.listdir(f"{data_dir}/{corpus}/")):
        fin = open(f"{data_dir}/{corpus}/{collection}", 'rb')
        encoded_message = fin.readlines()
        sender, content_type, boundary, split_indices = extract_metadata(encoded_message, corpus, collection)
        charset = "latin-1"
        
        if len(split_indices) > 2:
            for i in range(len(split_indices) - 1):
                x = split_indices[i]
                y = split_indices[i + 1]
                j = x + 1
                charset_found = False
                while j < y:
                    line = encoded_message[j].decode("latin-1").strip()
                    if len(line) == 0:
                        break
                    if "charset" in line:
                        charset = line.split("=")[1][1:-1]
                    j += 1
                print("--- EMAIL: --- ", collection, corpus, True)
                parser = MyHTMLParser()   
                parser.feed(b"".join(encoded_message[j+1:y]).decode(charset))
                print(parser.current_email)
        else:
            print("--- EMAIL: --- ", collection, corpus, False)
            parser = MyHTMLParser()   
            parser.feed(b"".join(encoded_message[split_indices[0]:split_indices[1]]).decode(charset))
            print(parser.current_email)
            
        fin.close()
        file_limit -= 1
        if file_limit == 0:
            break
        
    dir_limit -= 1
    if dir_limit == 0:
        break